Reformulação conforme implementada em "Lagrangian heuristics for the capacitated multi-plant lot sizing problem with multiple periods and items"

In [107]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math

# Leitura de instâncias

In [108]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1

    # Verifica se a matriz de demanda está na forma padrão
    num_demand_lines = 0
    for i in range(demand_start_line, len(lines)):
        if lines[i].strip():  # Se a linha não estiver vazia, contamos como linha de demanda
            num_demand_lines += 1
        else:
            break  # Interrompe se encontrar uma linha vazia (ou um bloco separado)
    
    # Calcula o multiplicador, caso tenha mais linhas do que períodos
    if num_demand_lines == periods * 2:
        multiplier = 2  # A matriz de demanda tem o dobro de linhas
    else:
        multiplier = 1  # A matriz de demanda tem o número esperado de linhas (T)
    
    # Leitura inicial das demandas
    for i in range(periods * multiplier):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)

    # print('Linha 1:', demand_matrix[0])
    # print('Linha 13:', demand_matrix[periods], '\n\n')
    
    # Agora, se multiplier == 2, precisamos combinar as linhas 1-12 com 13-24 corretamente
    if multiplier == 2:
        new_demand_matrix = []
        
        for i in range(periods):  # Para cada linha do primeiro bloco (1-12)
            combined_period_demand = []  # Lista única para armazenar a demanda combinada para o período i
            
            # Quebrar a linha i entre as plantas
            demands_first_part = demand_matrix[i]
            demands_second_part = demand_matrix[i + periods]  # Linha correspondente do segundo bloco
            
            # Calcular o número de elementos por planta
            elements_per_plant_first = len(demands_first_part) // num_plants
            elements_per_plant_second = len(demands_second_part) // num_plants
            
            # Concatenar as demandas para cada planta
            for j in range(num_plants):
                # Particionar a demanda de cada planta
                plant_demand_first = demands_first_part[j*elements_per_plant_first:(j+1)*elements_per_plant_first]
                plant_demand_second = demands_second_part[j*elements_per_plant_second:(j+1)*elements_per_plant_second]
                
                # Concatenar as demandas da planta j (primeiro e segundo blocos) e adicionar diretamente à lista única
                combined_period_demand.extend(plant_demand_first + plant_demand_second)
            
            # Adicionar a demanda ajustada (em formato de lista única) do período à nova matriz de demanda
            new_demand_matrix.append(combined_period_demand)
        
        # Substitui a matriz de demandas pela combinada e particionada por planta
        demand_matrix = new_demand_matrix
    
    # print('Linha 1 output após junção:', demand_matrix[0])

    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    # print(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods * multiplier
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver múltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [109]:
# Exemplo de uso
file_path = '../instancias/maria_desiree/NBB01246_1.dat'
data = read_dat_file(file_path)
display(data)

{'items': 6,
 'periods': 12,
 'num_plants': 4,
 'capacities': array([[2035, 2035, 2035, 2035, 2035, 2035, 2035, 2035, 2035, 2035, 2035,
         2035],
        [2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017,
         2017],
        [2101, 2101, 2101, 2101, 2101, 2101, 2101, 2101, 2101, 2101, 2101,
         2101],
        [1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872, 1872,
         1872]]),
 'production_time': array([[2.6, 4.3, 3.8, 1.2],
        [3.6, 1.7, 1.8, 3.6],
        [3. , 1.9, 1.9, 3.6],
        [1.9, 4.1, 4.4, 3.4],
        [4.8, 4.6, 4.6, 3.9],
        [4.8, 3.3, 4.2, 4. ]]),
 'setup_time': array([[12.7, 10.7, 27.4, 20. ],
        [10.7, 36.4, 36. , 31.9],
        [41.4, 42. , 48.2, 18.5],
        [40.6, 28.3, 37.3, 28.6],
        [29.5, 10.9, 40.7, 19.9],
        [42.5, 19.5, 22. , 14.4]]),
 'setup_cost': array([[59.5, 57.4, 19.5, 10. ],
        [80.3, 43.9, 30.3, 35.2],
        [ 7.7, 15.3, 65.2, 77.4],
        [ 9.3, 42.8, 79.5,  7.2],
  

# Modelagem

In [110]:
m = gp.Model('Lot-sizing Reformulation')

## Conjuntos

In [111]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

## Parâmetros

In [112]:
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

## Variáveis de decisão

In [113]:
# Quantidade produzida (i, j, t, k, u)
X = m.addVars(I, J, T, J, T, vtype=GRB.INTEGER, name='X')
# Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

## Função objetivo

In [114]:
expr1 + expr2

<gurobi.LinExpr: 1.7 <gurobi.Var *Awaiting Model Update*> + 1.9 <gurobi.Var *Awaiting Model Update*> + 2.1 <gurobi.Var *Awaiting Model Update*> + 2.3 <gurobi.Var *Awaiting Model Update*> + 2.5 <gurobi.Var *Awaiting Model Update*> + 2.7 <gurobi.Var *Awaiting Model Update*> + 2.9000000000000004 <gurobi.Var *Awaiting Model Update*> + 3.1 <gurobi.Var *Awaiting Model Update*> + 3.3 <gurobi.Var *Awaiting Model Update*> + 3.5 <gurobi.Var *Awaiting Model Update*> + 3.7 <gurobi.Var *Awaiting Model Update*> + 3.9000000000000004 <gurobi.Var *Awaiting Model Update*> + 1.92 <gurobi.Var *Awaiting Model Update*> + 2.12 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.52 <gurobi.Var *Awaiting Model Update*> + 2.7199999999999998 <gurobi.Var *Awaiting Model Update*> + 2.92 <gurobi.Var *Awaiting Model Update*> + 3.12 <gurobi.Var *Awaiting Model Update*> + 3.3200000000000003 <gurobi.Var *Awaiting Model Update*> + 3.52 <gurobi.Var *Awaiting Model Update*> + 3.71999999999

In [115]:
expr1 = sum(sum(sum(sum(sum(
    X[i, j, t, k, u] * (c[i, j] + min((u - t) * h[i, v] + r[j, v] + r[v, k] for v in J) if u >= t else 0) 
    for u in T) for k in J) for t in T) for j in J) for i in I)
expr2 = sum(sum(sum(s[i, j] * Z[i, j, t] for t in T) for j in J) for i in I)
m.setObjective(expr1 + expr2, sense=GRB.MINIMIZE)

## Restrições

In [116]:
# Balanço de estoque
m.addConstrs((sum(sum(X[i, j, t, k, u] for t in range(u + 1)) for j in J) == d[i, k, u] for i in I for k in J for u in T), name='restricao_balanco_estoque');

In [117]:
# Restrição que obriga setup
m.addConstrs((X[i, j, t, k, u] <= min(math.floor((cap[j, t] - f[i, j]) / b[i, j]), d[i, k, u]) * Z[i, j, t] for i in I for j in J for t in T for k in J for u in T)
             , name='restricao_setup');

In [118]:
# Restrição de capacidade
m.addConstrs((sum(f[i, j] * Z[i, j, t] + sum(sum(b[i, j] * X[i, j, t, k, u] for u in T) for k in J) for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

# Resolução

In [119]:
# Critérios de parada
m.Params.timelimit = 1800  # Tempo (default = inf)
# m.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)

# Otimização
m.optimize()

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14160 rows, 14112 columns and 49152 nonzeros
Model fingerprint: 0xa9d4475b
Variable types: 0 continuous, 14112 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 6370 rows and 6368 columns
Presolve time: 0.34s
Presolved: 7790 rows, 7744 columns, 30112 nonzeros
Variable types: 0 continuous, 7744 integer (332 binary)

Root relaxation: objective 5.467763e+04, 932 iterations, 0.08 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

In [120]:
# Número de períodos da instância
len(T)

12

In [121]:
# Número de fábricas
len(J)

4

In [122]:
# Número de produtos
len(I)

6

In [123]:
# Tempo de execução em segundos
m.Runtime

68.67599987983704

In [124]:
# Nome do modelo
m.ModelName

'Lot-sizing Reformulation'

In [125]:
# Valor da função objetivo da melhor solução encontrada (upper bound da minimização)
m.ObjVal

55025.23

In [126]:
# Lower bound da minimização
m.ObjBound

55022.48285208782

In [127]:
# Gap da solução
m.MIPgap

4.9925241787837867e-05

In [128]:
m.Status  # Ver https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes para interpretar códigos

2